In [1]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from enum import Enum
from copy import copy
from scipy.stats import norm
from scipy.cluster import hierarchy as sch
import pandas as pd
import numpy as np

import yfinance as yf

import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from _utils.core_functions import *
from _utils.portfoliohandcrafiting import *
from _utils.strategies.trend_simple_filter import *


DATA_DIR = "../_databases"

DEFAULT_DATE_FORMAT = "%Y-%m-%d"


In [2]:
#%% GET DATA
# # =================

def get_data_dict(instrument_list: list):

    all_data = dict(
        [
            (instrument_code, pd.read_csv((DATA_DIR + '/' + f'{instrument_code}' + '.csv'),index_col='index'))
            for instrument_code in instrument_list
        ]
    )

    adjusted_prices = dict(
        [
            (instrument_code, data_for_instrument.adjusted)
            for instrument_code, data_for_instrument in all_data.items()
        ]   
    )

    current_prices = dict(
        [
            (instrument_code, data_for_instrument.underlying)
            for instrument_code, data_for_instrument in all_data.items()
        ]
    )

    return adjusted_prices, current_prices

In [3]:
us_adjusted_prices, current_prices = get_data_dict(['sp500','us10'])
multipliers = dict(sp500=5, us10=1000)
risk_target_tau = 0.2

fx_series_dict = create_fx_series_given_adjusted_prices_dict(us_adjusted_prices,dict(sp500=1, us10=1))

capital = 1000000
idm = 1.5
instrument_weights = dict(sp500=0.5, us10=0.5)

std_dev_dict = calculate_variable_standard_deviation_for_risk_targeting_from_dict(
adjusted_prices=us_adjusted_prices,
current_prices=current_prices,
annualise_stdev=True,  ## can also be False if want to use daily price diff
use_perc_returns=True,  ## can also be False if want to use daily price diff
)

In [4]:

# 50% - 50%  US portfolio with EWMAC long only filter
us_average_position_contracts_dict = (
        calculate_position_series_given_variable_risk_for_dict(
            capital=capital,
            risk_target_tau=risk_target_tau,
            idm=idm,
            weights=instrument_weights,
            std_dev_dict=std_dev_dict,
            fx_series_dict=fx_series_dict,
            multipliers=multipliers,
        )
    )

us_position_contracts_filter_dict = calculate_position_dict_with_symmetric_trend_filter_applied(
        adjusted_prices_dict=us_adjusted_prices,
        average_position_contracts_dict=us_average_position_contracts_dict,
    )

us_position_contracts_longonly_dict = calculate_position_dict_with_trend_filter_applied(
        adjusted_prices_dict=us_adjusted_prices,
        average_position_contracts_dict=us_average_position_contracts_dict,
    )

us_perc_return_filter_dict = calculate_perc_returns_for_dict(
position_contracts_dict=us_position_contracts_filter_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=us_adjusted_prices,
)

us_perc_return_longonly_dict = calculate_perc_returns_for_dict(
position_contracts_dict=us_position_contracts_longonly_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=us_adjusted_prices,
)

us_perc_return_dict = calculate_perc_returns_for_dict(
position_contracts_dict=us_average_position_contracts_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=us_adjusted_prices,
)




us_port_filter_return = aggregate_returns(us_perc_return_filter_dict)
us_port_filter_return.sort_index(inplace=True)

us_port_longonly_return = aggregate_returns(us_perc_return_longonly_dict)
us_port_longonly_return.sort_index(inplace=True)

us_port_return = aggregate_returns(us_perc_return_dict)
us_port_return.sort_index(inplace=True)



In [5]:
# --- Plot retorno acumulado
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=us_port_filter_return.index,
        y=us_port_filter_return.cumsum()*100,
        name="EWMAC Portfolio - Long Short",
        line=dict(color="blue"),
    )
)

fig.add_trace(
    go.Scatter(
        x=us_port_longonly_return.index,
        y=us_port_longonly_return.cumsum()*100,
        name="EWMAC Portfolio - Long Only",
        line=dict(color="red"),
    )
)

fig.add_trace(
    go.Scatter(
        x=us_port_return.index,
        y=us_port_return.cumsum()*100,
        name="Portfolio without filter",
        line=dict(color="green"),
    )
)

fig.update_layout(
    title="50% SP500 Fut + 50% US10 Fut Portfolio",
    xaxis_title="Data",
    yaxis_title="Acumulated return (%)",
    template="plotly_white",
    showlegend=True,
    legend=dict(
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1),
    height=600,
    width=1000,
)

fig.show()

# --- Exibir estatísticas
print("Estatísticas de Desempenho")
print("Estatísticas Portfolio EWMAC Long and Short na Frequência Anual")
print(calculate_stats(us_port_filter_return, freq=YEAR))
print("================================")
print("Estatísticas Portfolio EWMAC Long Only Frequência Anual")
print(calculate_stats(us_port_longonly_return,freq=YEAR))
print("================================")
print("Estatísticas Portfolio sem EWMAC na Frequência Anual")
print(calculate_stats(us_port_return, freq=YEAR))


Estatísticas de Desempenho
Estatísticas Portfolio EWMAC Long and Short na Frequência Anual
{'ann_mean': 45.359396061915184, 'ann_std': 3.1618588481980896, 'sharpe': 14.345800441966292, 'skew': 0.3301357148434221, 'avg_drawdown': 0.021969953154384866, 'max_drawdown': 0.18921481813521956, 'quant_lower': 0.6108638011969315, 'quant_upper': 1.1910372532764422}
Estatísticas Portfolio EWMAC Long Only Frequência Anual
{'ann_mean': 47.579503820663184, 'ann_std': 3.0633586022690147, 'sharpe': 15.531810015785053, 'skew': 0.7418951765076701, 'avg_drawdown': 0.010849258145867532, 'max_drawdown': 0.1897868605720996, 'quant_lower': 0.675178081332698, 'quant_upper': 1.421817549424139}
Estatísticas Portfolio sem EWMAC na Frequência Anual
{'ann_mean': 49.799611579411156, 'ann_std': 4.275256714377516, 'sharpe': 11.648332464325959, 'skew': -0.25718198652478585, 'avg_drawdown': 0.025407764464484384, 'max_drawdown': 0.541110068938341, 'quant_lower': 1.8904168934615735, 'quant_upper': 1.0406914060284809}


In [6]:
br_adjusted_prices, current_prices = get_data_dict(['winfut','wdofut'])
multipliers = dict(winfut=5, wdofut=1000)
risk_target_tau = 0.2

fx_series_dict = create_fx_series_given_adjusted_prices_dict(br_adjusted_prices,dict(winfut=1, wdofut=1))

capital = 1000000
idm = 1.5
instrument_weights = dict(winfut=0.5, wdofut=0.5)

std_dev_dict = calculate_variable_standard_deviation_for_risk_targeting_from_dict(
adjusted_prices=br_adjusted_prices,
current_prices=current_prices,
annualise_stdev=True,  ## can also be False if want to bre daily price diff
use_perc_returns=True,  ## can also be False if want to bre daily price diff
)



# 50% - 50%  br portfolio with EWMAC long only filter
br_average_position_contracts_dict = (
        calculate_position_series_given_variable_risk_for_dict(
            capital=capital,
            risk_target_tau=risk_target_tau,
            idm=idm,
            weights=instrument_weights,
            std_dev_dict=std_dev_dict,
            fx_series_dict=fx_series_dict,
            multipliers=multipliers,
        )
    )

br_position_contracts_filter_dict = calculate_position_dict_with_symmetric_trend_filter_applied(
        adjusted_prices_dict=br_adjusted_prices,
        average_position_contracts_dict=br_average_position_contracts_dict,
    )

br_position_contracts_longonly_dict = calculate_position_dict_with_trend_filter_applied(
        adjusted_prices_dict=br_adjusted_prices,
        average_position_contracts_dict=br_average_position_contracts_dict,
    )

br_perc_return_filter_dict = calculate_perc_returns_for_dict(
position_contracts_dict=br_position_contracts_filter_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=br_adjusted_prices,
)

br_perc_return_longonly_dict = calculate_perc_returns_for_dict(
position_contracts_dict=br_position_contracts_longonly_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=br_adjusted_prices,
)

br_perc_return_dict = calculate_perc_returns_for_dict(
position_contracts_dict=br_average_position_contracts_dict,
fx_series=fx_series_dict,
multipliers=multipliers,
capital=capital,
adjusted_prices=br_adjusted_prices,
)




br_port_filter_return = aggregate_returns(br_perc_return_filter_dict)
br_port_filter_return.sort_index(inplace=True)

br_port_longonly_return = aggregate_returns(br_perc_return_longonly_dict)
br_port_longonly_return.sort_index(inplace=True)

br_port_return = aggregate_returns(br_perc_return_dict)
br_port_return.sort_index(inplace=True)






In [7]:
# --- Plot retorno acumulado
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=br_port_filter_return.index,
        y=br_port_filter_return.cumsum()*100,
        name="EWMAC Portfolio - Long Short",
        line=dict(color="blue"),
    )
)

fig.add_trace(
    go.Scatter(
        x=br_port_longonly_return.index,
        y=br_port_longonly_return.cumsum()*100,
        name="EWMAC Portfolio - Long Only",
        line=dict(color="red"),
    )
)

fig.add_trace(
    go.Scatter(
        x=br_port_return.index,
        y=br_port_return.cumsum()*100,
        name="Portfolio without filter",
        line=dict(color="green"),
    )
)

fig.update_layout(
    title="50% SP500 Fut + 50% US10 Fut Portfolio",
    xaxis_title="Data",
    yaxis_title="Acumulated return (%)",
    template="plotly_white",
    showlegend=True,
    legend=dict(
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1),
    height=600,
    width=1000,
)

fig.show()

# --- Exibir estatísticas
print("Estatísticas de Desempenho")
print("Estatísticas Portfolio EWMAC Long and Short na Frequência Anual")
print(calculate_stats(br_port_filter_return, freq=YEAR))
print("================================")
print("Estatísticas Portfolio EWMAC Long Only Frequência Anual")
print(calculate_stats(br_port_longonly_return,freq=YEAR))
print("================================")
print("Estatísticas Portfolio sem EWMAC na Frequência Anual")
print(calculate_stats(br_port_return, freq=YEAR))

Estatísticas de Desempenho
Estatísticas Portfolio EWMAC Long and Short na Frequência Anual
{'ann_mean': 4.147397592177511, 'ann_std': 2.239729754072966, 'sharpe': 1.8517401863485703, 'skew': 1.3283837697237821, 'avg_drawdown': 0.11423513153869043, 'max_drawdown': 0.31060674504977104, 'quant_lower': 0.6424727386529744, 'quant_upper': 13.476636456401947}
Estatísticas Portfolio EWMAC Long Only Frequência Anual
{'ann_mean': 9.113909970365281, 'ann_std': 1.576664314458058, 'sharpe': 5.7805012054820155, 'skew': 0.9258897422086866, 'avg_drawdown': 0.050128695365566134, 'max_drawdown': 0.15912629258903327, 'quant_lower': 0.439081855820993, 'quant_upper': 1.4952221385438045}
Estatísticas Portfolio sem EWMAC na Frequência Anual
{'ann_mean': 14.08042234855305, 'ann_std': 2.0005049381946907, 'sharpe': 7.03843418715057, 'skew': 0.9862544076184095, 'avg_drawdown': 0.04620281739164987, 'max_drawdown': 0.23618220076495366, 'quant_lower': 0.5306260805877093, 'quant_upper': 1.6664968959608026}
